In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

ticker = 'AAPL'
filename = '../../../../Data_Store/Yah_Daily_Prices/Daily_Price_Data/{}.pk'.format(ticker)
with open(filename, 'rb') as file:
    # Change Index to timestamp for original Polygon Data
    # Change Index to Date for Yah Finance Data
    data = pd.DataFrame(pickle.load(file)).set_index('Date', drop=True)

##### Calculating the EMA requires one more observation than the SMA. Suppose that you want to use 20 days as the number of observations for the EMA. Then, you must wait until the 20th day to obtain the SMA. On the 21st day, you can then use the SMA from the previous day as the first EMA for yesterday.

data = data.fillna(0)
smoothing = 2
multiplier = {}

for period in [12, 26]:
    multiplier["{}".format(period)] = float(smoothing/(1+period))
    data['sma_{}'.format(period)] = data['Adj Close'].rolling(window=period).mean()
    data['ema_{}'.format(period)] = [data['Adj Close'][i]*multiplier["{}".format(period)] + data['sma_{}'.format(period)][i-1]*(1 - multiplier["{}".format(period)]) for i in range(0, len(data))]
    data['ema_{}'.format(period)] = [data['Adj Close'][i]*multiplier["{}".format(period)] + data['ema_{}'.format(period)][i-1]*(1 - multiplier["{}".format(period)]) for i in range(0, len(data))]
   
    
data['MACD'] = data['ema_12'] - data['ema_26']
sig_multiplier = float(smoothing/(1+9))
data['sma_MACD'] = data['MACD'].rolling(window=9).mean()
data['signal_line'] = [data['MACD'][i]*sig_multiplier + data['sma_MACD'][i]*(1 - sig_multiplier) for i in range(0, len(data))]
data['signal_line'] = [data['MACD'][i] * sig_multiplier + data['signal_line'][i-1]*(1 - sig_multiplier) for i in range(0, len(data))]

In [2]:
data = data.fillna(0)
smoothing = 2
multiplier = {}
ema = {}

for period in [12, 26]:
    multiplier["{}".format(period)] = float(smoothing/(1+period))
    data['sma_{}'.format(period)] = data['Adj Close'].rolling(window=period).mean()
    place_holder = data['Adj Close'][period]*multiplier["{}".format(period)] + data['sma_{}'.format(period)][period-1]*(1 - multiplier["{}".format(period)])
    ema["{}".format(period)] = []
    for i in range(0, len(data)):
        if i < period:
            ema["{}".format(period)].append(np.nan)
        elif i == period:
            ema["{}".format(period)].append(data['Adj Close'][i]*multiplier["{}".format(period)] + place_holder*(1-multiplier["{}".format(period)]))
        elif i > period:
            ema["{}".format(period)].append(data['Adj Close'][i]*multiplier["{}".format(period)] + ema["{}".format(period)][i-1]*(1 - multiplier["{}".format(period)]))
        else:
            print("Fuck.")
   
    data["ema_{}".format(period)] = ema["{}".format(period)]
    
data['MACD'] = data['ema_12'] - data['ema_26']
sig_multiplier = float(smoothing/(1+9))
print(sig_multiplier)
data['sma_MACD'] = data['MACD'].rolling(window=9).mean()
place_holder = data['MACD'][9+26]*sig_multiplier + data['sma_MACD'][9+26-1]*(1 - sig_multiplier)
print(place_holder)
#data['signal_line'] = [data['MACD'][i]*sig_multiplier + data['sma_MACD'][i]*(1 - sig_multiplier) for i in range(0, len(data))]
sig_ema = []
for i in range(0, len(data)):
    if i < (9+26):
        sig_ema.append(np.nan)
    elif i == (9+26):
        sig_ema.append(data['MACD'][i]*sig_multiplier + place_holder*(1 - sig_multiplier))
    elif i > (9+26):
        sig_ema.append(data['MACD'][i]*sig_multiplier + sig_ema[i-1]*(1 - sig_multiplier))
data['signal_line'] = sig_ema

0.2
0.024917209686091298


In [3]:
data.head()

,High,Low,Open,Close,Volume,Adj Close,sma_12,ema_12,sma_26,ema_26,MACD,sma_MACD,signal_line
Date,,,,,,,,,,,,,
2000-01-03,1.004464,0.907924,0.936384,0.999442,535796800.0,0.856227,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,0.987723,0.903460,0.966518,0.915179,512377600.0,0.784038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,0.987165,0.919643,0.926339,0.928571,778321600.0,0.795511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,0.955357,0.848214,0.947545,0.848214,767972800.0,0.726669,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,0.901786,0.852679,0.861607,0.888393,460734400.0,0.761090,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data[-25:]

,High,Low,Open,Close,Volume,Adj Close,sma_12,ema_12,sma_26,ema_26,MACD,sma_MACD,signal_line
Date,,,,,,,,,,,,,
2021-03-25,121.660004,119.000000,119.540001,120.589996,98844700.0,120.589996,122.034999,121.949408,122.978076,123.795926,-1.846518,-2.086867,-2.096448
2021-03-26,121.480003,118.919998,120.349998,121.209999,94071200.0,121.209999,122.137499,121.835653,122.651153,123.604376,-1.768723,-1.975985,-2.030903
2021-03-29,122.580002,120.730003,121.650002,121.389999,80819200.0,121.389999,122.089999,121.767091,122.324999,123.440348,-1.673257,-1.903643,-1.959374
2021-03-30,120.400002,118.860001,120.110001,119.900002,85671900.0,119.900002,121.995832,121.479846,122.090384,123.178100,-1.698254,-1.868348,-1.907150
2021-03-31,123.519997,121.150002,121.650002,122.150002,118323800.0,122.150002,121.842499,121.582947,121.947692,123.101944,-1.518998,-1.805044,-1.829520
2021-04-01,124.180000,122.489998,123.660004,123.000000,75089100.0,123.000000,121.628333,121.800955,121.857307,123.094393,-1.293438,-1.708198,-1.722303
2021-04-05,126.160004,123.070000,123.870003,125.900002,88651200.0,125.900002,121.723333,122.431578,122.046153,123.302216,-0.870638,-1.590606,-1.551970
2021-04-06,127.129997,125.650002,126.500000,126.209999,80171300.0,126.209999,122.196666,123.012873,122.236538,123.517607,-0.504734,-1.447821,-1.342523
2021-04-07,127.919998,125.139999,125.830002,127.900002,83466700.0,127.900002,122.855833,123.764739,122.240769,123.842229,-0.077490,-1.250228,-1.089516
